<a href="https://www.kaggle.com/code/kajuyerim/lora-guide-on-llama3-1-8b-instruct?scriptVersionId=191369410" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# LoRA Guide About Sentimental Analysis on Financial Domain Using Llama3.1 8b-instruct
---

<font size="5">**What is LoRA?**</font> 
* <font size="3"> Low Rank Adaptation: Freezing the pretrained model weights and injecting trainable rank decomposition matrices into each layer of the Transformer architecture.</font> 
* <font size="3"> Main purpose is to lower the dimensions of the matrices. </font> 
* <font size="3"> Instead of updating all weights of a model, we only update the injected low rank matrices.</font> 

<font size="5">**Why Use LoRA?**</font> 
* <font size="3"> Greatly reducing the number of trainable parameters (up to 10000 times).</font> 
* <font size="3"> Reducing the GPU memory requirement (up to 3 times).</font> 

---
<font size="4">**! This guide is based on the LoRA paper (LoRA: Low-Rank Adaptation for Neural Networks)** (https://arxiv.org/abs/2106.09685#).</font>   
<font size="4">**! Any mistakes between this guide and the paper are due to my interpretation on the material.**</font>  

---


In [3]:

!pip install -U datasets transformers

  Using cached transformers-4.43.4-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.43.4-py3-none-any.whl (9.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Can't uninstall 'transformers'. No files were found to uninstall.


### Preparing the Dataset
<font size="3"> We needed to shuffle the database because it is sorted, we need it shuffled. (Dataset: https://huggingface.co/datasets/takala/financial_phrasebank)


In [4]:
# Re-import necessary libraries since the code execution state was reset.
from datasets import load_dataset

# Load and shuffle the dataset with a specified configuration
dataset = load_dataset("takala/financial_phrasebank", "sentences_allagree", split='train')
shuffled_dataset = dataset.shuffle(seed=32)

# Map numerical labels to string labels
label_mapping = {0: "negative", 1: "neutral", 2: "positive"}

# Format examples into "sentence" - "sentiment" format
formatted_examples = [
    f'"{example["sentence"]}" - "{label_mapping[example["label"]]}"' for example in shuffled_dataset
]

# Print the first 5 formatted examples
for example in formatted_examples[:5]:
    print(example)


The repository for takala/financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/takala/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

"The value of the order is USD 2.2 mn ." - "neutral"
"YIT lodged counter claims against Neste Oil totaling some EUR25m , primarily based on work carried out under the contract and additional costs incurred due to prolongation of the project ." - "negative"
"Rohwedder Group is an automotive supplies , telecommunications and electronics industry provider for customers in Europe , North America and Asia ." - "neutral"
"Nokia was up 0.12 pct to 16.70 eur after kicking off the morning in negative territory ." - "positive"
"The Group owns and operates a fleet of more than 800dwt , while container capacity is 17,000 TEUs , and manages a diversified fleet of its own railway rolling stock of over 17,000 units ." - "neutral"


In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
!nvcc --version


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [8]:
import torch

print(torch.cuda.is_available())
print(torch.__version__)


True
2.1.2


In [9]:
import transformers
import torch

print(torch.cuda.is_available())
print(torch.__version__)

base_model = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
pipeline = transformers.pipeline(
    "text-generation",
    model=base_model,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)
outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])



True
2.1.2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


RuntimeError: cutlassF: no kernel found to launch!